In [ ]:
# Importar bibliotecas necesarias
import pandas as pd
import mysql.connector

In [ ]:

# Leer el archivo CSV
df = pd.read_csv('../data/requests.csv')


# Mostrar las primeras filas para verificar los datos
df.head()

# Transformaciones con Pandas

In [ ]:

# Convertir la fecha de la solicitud a formato datetime
df['Fecha'] = pd.to_datetime(df['F.solicitud'], errors='coerce')

# Crear columnas nuevas de mes y año
df['mes'] = df['Fecha'].dt.month
df['año'] = df['Fecha'].dt.year


Cálculo de indicadores

In [ ]:
# KPI Ejemplo: ¿Resoluciones rápidas?
df['resolucion_rapida'] = df['Estado'].apply(lambda x: 1 if x == 'resuelto en menos de 5 días' else 0)

# Ver el dataframe con las nuevas columnas y KPIs
df.head()

In [ ]:
# Conexión a MySQL
DB_CONFIG = {
    'host': 'localhost',
    'user': 'user',
    'password': 'password',
    'database': 'data_db'
    }

In [ ]:
def load_data_to_mysql(df):
    connection = mysql.connector.connect(
        host=DB_CONFIG['host'],
        user=DB_CONFIG['user'],
        password=DB_CONFIG['password'],
        database=DB_CONFIG['database']
    )
    cursor = connection.cursor()

    # Insertar los datos procesados en la tabla MySQL
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO solicitudes_procesadas 
            (tipo, estado, fecha_solicitud, origen, unidad, provincia, referencia, mes, año, resolucion_rapida) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['Tipo'], row['Estado'], row['Fecha'], row['Origen'], row['Unidad'], 
            row['Provincia'], row['Referencia'], row['mes'], row['año'], row['resolucion_rapida']
        ))

    # Confirmar la transacción
    connection.commit()
    cursor.close()
    connection.close()
    print("Datos cargados en MySQL correctamente.")


In [ ]:

# Ejecutar la función para cargar los datos en MySQL
load_data_to_mysql(df)